<a href="https://colab.research.google.com/github/kiyarashnd/promptops-project/blob/main/PromptOps_QC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PromptOps Quality Control Demo (Google Colab)
پیاده‌سازی عملی برای مقاله «کنترل کیفی خروجی‌های LLM مبتنی بر PromptOps».

**رویکردها:** Git-native PromptOps و PromptLayer (اختیاری)
**ابزارها:** LangChain + MLflow

## 1) نصب وابستگی‌ها

In [1]:
!pip -q install mlflow>=2.14 pandas pyyaml jinja2 pydantic langchain langchain-openai promptlayer

## 2) تنظیم کلیدها

In [2]:

import os
os.environ["OPENAI_API_KEY"] = "PASTE_YOUR_KEY"
# اختیاری:
# os.environ["PROMPTLAYER_API_KEY"] = "PASTE_YOUR_KEY"


## 3) Dataset نمونه (متن عمومی فارسی)

In [3]:

import pandas as pd
texts = [
"امروز صبح در محله ما برق برای دو ساعت قطع شد.",
"دیروز در مدرسه درباره اهمیت بازیافت صحبت شد.",
"یک فروشگاه آنلاین بسته را دیرتر از وعده ارسال کرد.",
"خبر علمی می‌گفت خواب کافی تمرکز را بهتر می‌کند.",
"در شبکه‌های اجتماعی شایعه‌ای بدون منبع پخش شد."
]
df = pd.DataFrame({"id": range(1, len(texts)+1), "input_text": texts})
df


,id,input_text
0,1,امروز صبح در محله ما برق برای دو ساعت قطع شد.
1,2,دیروز در مدرسه درباره اهمیت بازیافت صحبت شد.
2,3,یک فروشگاه آنلاین بسته را دیرتر از وعده ارسال ...
3,4,خبر علمی می‌گفت خواب کافی تمرکز را بهتر می‌کند.
4,5,در شبکه‌های اجتماعی شایعه‌ای بدون منبع پخش شد.


## 4) پرامپت نسخه‌ دار (Git-native)

In [4]:

import yaml, textwrap
from pathlib import Path

base = Path("/content/promptops")
(base/"prompts_git"/"extractor").mkdir(parents=True, exist_ok=True)

yaml_content = textwrap.dedent("""
messages:
  - role: system
    content: |
      فقط یک JSON معتبر برگردان با فیلدهای:
      topic, entities, sentiment, summary_fa
  - role: user
    content: |
      متن:
      {{ input_text }}
""")
(base/"prompts_git"/"extractor"/"v1.yaml").write_text(yaml_content, encoding="utf-8")


196

## 5) QC و Prompt Manager

In [5]:

from pydantic import BaseModel, Field
from typing import List
from jinja2 import Template
import json

class Extraction(BaseModel):
    topic: str
    entities: List[str]
    sentiment: str
    summary_fa: str

class GitPromptManager:
    def __init__(self, base_dir):
        self.base_dir = Path(base_dir)
    def get(self, name, version):
        return yaml.safe_load((self.base_dir/name/f"{version}.yaml").read_text())['messages']
    def render(self, messages, vars):
        return [{"role": m["role"], "content": Template(m["content"]).render(**vars)} for m in messages]

def qc_validate(txt):
    try:
        Extraction.model_validate(json.loads(txt))
        return True
    except Exception:
        return False


## 6) اجرای LangChain + MLflow

In [6]:

import mlflow
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

def call_llm(messages):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    lc=[]
    for m in messages:
        lc.append(SystemMessage(content=m["content"]) if m["role"]=="system"
                  else HumanMessage(content=m["content"]))
    return llm.invoke(lc).content

mlflow.set_experiment("promptops-qc-colab")
git_mgr = GitPromptManager(base/"prompts_git")

with mlflow.start_run(run_name="git_promptops"):
    okc=0
    for _,r in df.iterrows():
        msgs = git_mgr.render(git_mgr.get("extractor","v1"), {"input_text": r["input_text"]})
        out = call_llm(msgs)
        ok = qc_validate(out)
        mlflow.log_metric("qc_pass", int(ok), step=int(r["id"]))
        okc += int(ok)
    mlflow.log_metric("qc_pass_rate", okc/len(df))
print("Done")


2026/02/04 10:37:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/04 10:37:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/04 10:37:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/04 10:37:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/04 10:37:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/04 10:37:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/04 10:37:35 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/04 10:37:35 INFO mlflow.store.db.utils: Updating database tables
2026/02/04 10:37:35 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/04 10:37:35 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/04 10:37:36 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/02/04 10:3

Done
